In [1]:
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph

from config import CFG_SERVICE
from src.utils import get_langchain_azure_chat_llm

LLM = get_langchain_azure_chat_llm()
URL = CFG_SERVICE.neo4j.uri
USERNAME = CFG_SERVICE.neo4j.username
PASSWORD = CFG_SERVICE.neo4j.password

graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)

In [2]:
# If the schema of database changes, you can refresh the schema information needed to generate Cypher statements
# graph.refresh_schema()
print(graph.schema)

Node properties:
Person {name: STRING, born: INTEGER}
Movie {tagline: STRING, title: STRING, released: INTEGER}
Relationship properties:
ACTED_IN {roles: LIST}
REVIEWED {summary: STRING, rating: INTEGER}
The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:PRODUCED]->(:Movie)
(:Person)-[:WROTE]->(:Movie)
(:Person)-[:FOLLOWS]->(:Person)
(:Person)-[:REVIEWED]->(:Movie)


In [3]:
from langchain_neo4j.chains.graph_qa.cypher import construct_schema

graph_schema = construct_schema(
    graph.get_structured_schema, include_types=[], exclude_types=[]
)
print(graph_schema)

Node properties are the following:
Person {name: STRING, born: INTEGER},Movie {tagline: STRING, title: STRING, released: INTEGER}
Relationship properties are the following:
ACTED_IN {roles: LIST},REVIEWED {summary: STRING, rating: INTEGER}
The relationships are the following:
(:Person)-[:ACTED_IN]->(:Movie),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:PRODUCED]->(:Movie),(:Person)-[:WROTE]->(:Movie),(:Person)-[:FOLLOWS]->(:Person),(:Person)-[:REVIEWED]->(:Movie)


In [4]:
chain = GraphCypherQAChain.from_llm(
    LLM,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps=True,
    # return_direct,
    # use_function_response=True,
    # top_k=5,
)

In [12]:
res = chain.invoke({"query": "Tony Scott 감독의 영화에 출연한 배우 알려줘"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person)
WHERE p.name = 'Tony Scott'
RETURN a.name

Full Context:
[{'a.name': 'Val Kilmer'}, {'a.name': 'Meg Ryan'}, {'a.name': 'Tom Skerritt'}, {'a.name': 'Kelly McGillis'}, {'a.name': 'Tom Cruise'}, {'a.name': 'Anthony Edwards'}]

> Finished chain.


In [13]:
res

{'query': 'Tony Scott 감독의 영화에 출연한 배우 알려줘',
 'result': 'Val Kilmer, Meg Ryan, Tom Skerritt, Kelly McGillis, Tom Cruise, Anthony Edwards는 Tony Scott 감독의 영화에 출연한 배우들입니다.',
 'intermediate_steps': [{'query': "cypher\nMATCH (p:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person)\nWHERE p.name = 'Tony Scott'\nRETURN a.name\n"},
  {'context': [{'a.name': 'Val Kilmer'},
    {'a.name': 'Meg Ryan'},
    {'a.name': 'Tom Skerritt'},
    {'a.name': 'Kelly McGillis'},
    {'a.name': 'Tom Cruise'},
    {'a.name': 'Anthony Edwards'}]}]}

In [14]:
res = chain.invoke(
    {"query": "Tony Scott이 감독을 하고 Tom Cruise가 출연한 영화가 뭐야?"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Tony Scott'})-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person {name: 'Tom Cruise'}) RETURN m.title, m.tagline, m.released
Full Context:
[{'m.title': 'Top Gun', 'm.tagline': 'I feel the need, the need for speed.', 'm.released': 1986}]

> Finished chain.


In [11]:
res

{'query': 'Top Gun의 감독은 누구인가요?',
 'result': 'Top Gun의 감독은 Tony Scott입니다.',
 'intermediate_steps': [{'query': 'MATCH (p:Person)-[:DIRECTED]->(m:Movie {title: "Top Gun"}) RETURN p.name'},
  {'context': [{'p.name': 'Tony Scott'}]}]}